<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-2/partA/partA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Before executing the code, allocate GPU in Colab from: Edit -> Notebook Settings -> Hardware accelerator -> GPU

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
import random
import os
from google.colab import drive
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing  
from keras.layers import LeakyReLU
from keras.models import Model



In [ ]:
#To access content from google drive
drive.mount('/content/gdrive')

In [ ]:
#Todo- first rename the file in drive
#Use this link: https://drive.google.com/drive/folders/1yu5DuYlM_cvV5qwIKKnl3sHRG_G41aLB?usp=sharing
#To make copy of pickled data and save it folder name 'Raw_data350'

with open('/content/gdrive/My Drive/Raw_data350/xtrain','rb') as xT:
  xTrain = pickle.load(xT)

with open('/content/gdrive/My Drive/Raw_data350/ytrain','rb') as yT:
  yTrain = pickle.load(yT)

with open('/content/gdrive/My Drive/Raw_data350/xtest','rb') as xV:
  xTest = pickle.load(xV)

with open('/content/gdrive/My Drive/Raw_data350/ytest','rb') as yV:
  yTest = pickle.load(yV)

In [ ]:
xplot=[]
yplot=[]
x=10
for i in range(30):
  xplot.append(xTest[x])
  yplot.append(yTest[x])
  x=x+25


In [ ]:
len(yTest)

In [ ]:
xTrain, xVal, yTrain, yVal= train_test_split(xTrain, yTrain, test_size=.1)

In [ ]:
len(xTrain)

In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Train data

for i in range(len(xTrain)):

  xTrain[i]=np.array(xTrain[i])
  
  if(xTrain[i].ndim==2):
    del xTrain[i]
    del yTrain[i]

xTrain=np.array(xTrain)


In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Validation data

for i in range(len(xVal)):

  xVal[i]=np.array(xVal[i])
  
  if(xVal[i].ndim==2):
    del xVal[i]
    del yVal[i]

xVal=np.array(xVal)

In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Train data

for i in range(len(xTest)):

  xTest[i]=np.array(xTest[i])
  
  if(xTest[i].ndim==2):
    del xTest[i]
    del yTest[i]

xTest=np.array(xTest)
type(xTrain)

In [ ]:
# Normalising xTrain
for i in range(len(xTrain)):
  # xTrain[i]= xTrain[i].astype('float32')
  xTrain[i]= xTrain[i]/255.0

In [ ]:
# Normalising xVal
for i in range(len(xVal)):

  xVal[i]=xVal[i].astype('float32')
  xVal[i]=xVal[i]/255.0

In [ ]:
# Normalising xTest
for i in range(len(xTest)):

  xTest[i]=xTest[i].astype('float32')
  xTest[i]=xTest[i]/255.0

In [ ]:
# Assigning the string labels of yTest and yTrain to integers
pple = preprocessing.LabelEncoder()
yTest=pple.fit_transform(yTest)
yVal=pple.fit_transform(yVal)
yTrain=pple.fit_transform(yTrain)

In [ ]:
#Change the integer y to one-hot vector
def to_categorical(y, n_class):
    return np.eye(n_class, dtype='uint8')[y]

In [ ]:
# getting the categorical value of yTrain and yTest using to_categorical() function
yTrain_cat=[]
for i in range(len(yTrain)):
  cat_d=to_categorical(int(yTrain[i]),10)
  yTrain_cat.append(cat_d)

yVal_cat=[]
for i in range(len(yVal)):
  cat_d=to_categorical(int(yVal[i]),10)
  yVal_cat.append(cat_d)

yTest_cat=[]
for i in range(len(yTest)):
  cat_d=to_categorical(int(yTest[i]),10)
  yTest_cat.append(cat_d)

In [ ]:
#changing the yTrain_cat and yTest_cat to numpy array 

yTrain_cat=np.array(yTrain_cat)
yVal_cat=np.array(yVal_cat)
yTest_cat=np.array(yTest_cat)

In [ ]:
print(yTrain_cat.shape)
print(yVal_cat.shape)
print(yTest_cat.shape)

In [ ]:
!pip install wandb

In [ ]:
import wandb
!wandb login

In [ ]:
sweep_config_temp={
  "name": "DL Assign-2-partA-final1",
  "method": "random",
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  "parameters": {
        "num_filters": {
            "values": [64,128,256]
        },
        "filter_organisation":{
            "values":['same','half']
        },
        "activationFunction":{
            "values":['leakyRelu','Relu']  
        },
        "data_augumentation":{
            "values":['No']  
        },
        "filterSize":{
            "values": [(2,2),(3,3),(4,4)]  
        },
        "dropout":{
            "values":[0.2,0.3,0.5]
        },
        "batch_normalization":{
            "values":['Yes', 'No']
        },
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config_temp, entity="cs20m014", project="DL Assign-2-partA-final")

In [ ]:
hyperparameter_defaults = dict(
    num_filters=64,
    filter_organisation='same',
    activationFunction='leakyRelu',
    data_augumentation='No',
    filterSize=(3,3),
    dropout=0.5,
    batch_normalization= 'Yes',
    )

In [ ]:
def model_run(eps, filter_size, mul_filter, drop, fSize, actv, batchNorm):

  current_filter_size=filter_size

  activationFun='relu'

  if actv=='leakyRelu':
    activationFun=LeakyReLU(alpha=0.1)

  model = Sequential()

  if batchNorm=='Yes':
    model.add(BatchNormalization(input_shape=(350, 350, 3)))

  

  model.add(Conv2D(current_filter_size, fSize, activation=activationFun, input_shape=(350, 350, 3)))
  model.add(MaxPooling2D((2, 2)))

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))

  if batchNorm=='Yes':
    model.add(BatchNormalization())

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
 
  model.add(MaxPooling2D((3, 3)))
  model.add(Flatten())
  model.add(Dropout(drop))


  model.add(Dense(1024, activation=activationFun))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  history = model.fit(xTrain,yTrain_cat,epochs = eps , validation_data = (xVal, yVal_cat))

  wandb.log({
        
        "Train Loss": history.history['loss'][eps-1],
        "Train Acc": history.history['accuracy'][eps-1],
        "Valid Loss":history.history['val_loss'][eps-1],
        "Valid Acc": history.history['val_accuracy'][eps-1]})
  return history.history['val_accuracy'][eps-1]
   

In [ ]:
def train():
  wandb.init(config=hyperparameter_defaults)
  config = wandb.config

  n_filters=config.num_filters
  multiplier=0

  if config.filter_organisation=='same':
    multiplier=1
  elif config.filter_organisation=='half':
    multiplier=0.5
  elif config.filter_organisation=='double':
    multiplier=2
  
  drop=config.dropout
  fSize=config.filterSize
  activation=config.activationFunction
  batchNorm = config.batch_normalization
  accuracy = model_run(5, n_filters, multiplier,drop,fSize, activation, batchNorm)
  

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
#Sukanya's part